In [9]:
%load_ext autoreload
%matplotlib inline

In [23]:
%autoreload 2

import sys 
import pandas as pd
import numpy as np
import datetime
from datetime import timezone
import pytz
import random
# from pycoingecko import CoinGeckoAPI
import statsmodels.tsa.stattools as ts 
import matplotlib.pyplot as plt

# Custom imports
from cointegridy.src.classes.basket import Basket
from cointegridy.src.classes.coin import Coin
from cointegridy.src.classes.processor import Processor
from cointegridy.src.classes.data_loader import DataLoader, TreeLoader
from cointegridy.src.classes.Time import Time
import cointegridy.src.utils.stats as stats
import cointegridy.src.utils.transforms as transform

In [24]:
###############
## CONSTANTS ##
###############

NAT_TZ = 'America/Los_Angeles'
NAT_TZOBJ = pytz.timezone(NAT_TZ)

TRG_TZOBJ = timezone.utc

PORT_IDS = ['bitcoin', 'bitcoin-cash', 'ethereum', 'litecoin']
start_date, end_date = (2020,1,1), (2021,10,31)

NAT_CURR = 'usd'

LOOKBACKS = [7, 14, 30]
ROLL_WIND = LOOKBACKS[1]

In [28]:
##################
## LOCAL DRIVER ##
##################

treeloader = TreeLoader()
coin_ids = treeloader.pc.get_api_ids()
# coin_ids = dataloader.get_ids()

start, stop, step = Time.date_to_Time(2021, 11, 1), Time.date_to_Time(2021, 12, 1), '6h'

example_id = coin_ids[0]
example_prices = treeloader[example_id][start:stop:step]

print(example_prices)

<generator object TreeSymbolLoader.__getitem__ at 0x7f87c2f33850>


# Tests
Test cointegration of coin basket: {coins} = {c_1,...,c_n}
ie. Check that the set {coins} is cointegrated using Engle-Granger two-step test
This means there is some STATIONARY linear combination of {coins}

<ol>
    <li>Check that c_i is order-1 integrable</li>
    <li>Create basket of coins, find linear regression.</li>
    <li>Verify spread of basket is cointegrated.</li>
    <li>Define strategy for basket trades.</li>
</ol>

#### Concerns
<ul>
    <li>Should we normalize data before running our regression?</li>
</ul>

#### Conclusions
<ul>
    <li>OLS and Linear Regression yield the same coefficients</li>
    <li>While we usually assume stock data is I(1), it sometimes isn't (eg. Litecoin, BTC-Cash)</li>
    <li>Normalization might help us find a regression line that does cointegrate the portfolio</li>
    <li>We should run this test on different coins in different sized baskets.</li>
</ul>

In [ ]:
############
## TEST 1 ##
############

# print("Normalizing...")
# scaler = MinMaxScaler()
# scaler.fit(X, y)
# X_scaled = scaler.transform(X) # TODO: Finish normalization
# X_scaled = pd.DataFrame(X_scaled/X_scaled[0], index=X.index) 
# X_scaled.columns = X.columns

print("Verifying coin price stationarity...")
coins = []
for coin_name in data:
    coin = Coin(coin_name)
    result = coin.is_good(data[coin_name], start_date, end_date)
    if result:
        coins.append(coin)
    else:
        print(coin_name, "is not I(1)")

print([coin.name_ for coin in coins], " are stationary \n")
        
basket = Basket(coins, target='bitcoin', processor=processor)
basket.fit(data)
spread = basket.find_spread(data)
is_cointegrated = basket.is_coint(spread)
print("\n")
print("Cointegration test returned:", is_cointegrated)

short_entry, long_entry = basket.strat(spread)

Verifying coin price stationarity...
['bitcoin', 'bitcoin-cash', 'ethereum', 'litecoin']  are stationary 



TypeError: __init__() got an unexpected keyword argument 'processor'

# Explore Exchanges on CoinGecko

In [ ]:
exchanges = processor.cg.get_exchanges_list()
print("There are", len(exchanges), "exchanges.")

by_24hr_vol = sorted(exchanges, key=lambda d: d['trade_volume_24h_btc'], reverse=True) 
print("Top 3 by 24 trading volume:", [exchange['id'] for exchange in by_24hr_vol[:3]])
by_trust_score = sorted(exchanges, key=lambda d: d['trust_score_rank'])
print("Top 3 by trust score:", [exchange['id'] for exchange in by_trust_score[:3]])

For now just use the highest 100 market cap coins on coingecko

In [ ]:
COLS = ['id', 'current_price', 'high_24h', 'low_24h', 'market_cap', 'total_volume']

coin_market = processor.cg.get_coins_markets(vs_currency=NAT_CURR)
df_market = pd.DataFrame(coin_market, columns=COLS)
df_market.set_index('id', inplace=True)

In [ ]:
df_market

From binomial theorem there are $2^{100}$ baskets so we need to find a way of filtering.

<b>There are also too many coins to pull all of them at once! We might have to create and host a database that contains updated data. We can pull around 50 or so coins at once.</b>

### Strategy
Take Pearson (Spearman?) correlation of dataframe to filter out some coins.

Idea:
We could train a model to take price data and try to spit out which coins are the most likely to be cointegrated. I think this could be some kind of 

In [ ]:
# Create portfolio dataframe
port_val = processor.create_portfolio(df_market[:50].index, start_date, end_date)
data = processor.data
# Write to CSV

In [ ]:
corr = data.corr();corr

In [ ]:
print("Verifying coin price stationarity...")
coins = []
for coin_name in data:
    coin = Coin(coin_name)
    result = coin.is_good(data[coin_name], start_date, end_date)
    if result:
        coins.append(coin)
    else:
        print(coin_name, "is not I(1)")

print([coin.name_ for coin in coins], " are stationary \n")

In [ ]:
coinNames = [coin.name_ for coin in coins]
df = data[coinNames].corr()
corrList = df.values.flatten()
plt.hist(corrList,bins=40)

In [ ]:
df.values[[np.arange(df.shape[0])]*2] = 0
plt.hist(df.corr().values.flatten(),bins=40)